# SWIFTCON 2024 - DMO Example

This notebook takes you through an example of running a dark matter only simulation and creating halo catalogues.

Created by Dr. Victor J. Forouhar Moreno PhD (forouhar@strw.leidenuniv.nl) & Ron (mcgibbon@strw.leidenuniv.nl)

#### Setup on cosma

This cell sets up dependencies for those running on cosma

In [1]:
module purge
module load python/3.12.4 gnu_comp/14.1.0 openmpi/5.0.3 parallel_hdf5/1.12.3
module load fftw gsl parmetis
venv_name="$(echo $HOME | sed 's/home/apps/')/swift_workshop_env"
python="$venv_name/bin/python3"

# monofonIC - Generate Initial Conditions



In [7]:
./getIC.sh

--2024-09-15 19:06:19--  http://virgodb.cosma.dur.ac.uk/swift-webstorage/ICs/small_cosmo_volume.hdf5
Resolving virgodb.cosma.dur.ac.uk (virgodb.cosma.dur.ac.uk)... 129.234.196.27
Connecting to virgodb.cosma.dur.ac.uk (virgodb.cosma.dur.ac.uk)|129.234.196.27|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8396888 (8.0M)
Saving to: ‘small_cosmo_volume.hdf5’

small_cosmo_volume. 100%[===================>]   8.01M  18.8MB/s    in 0.4s    

2024-09-15 19:06:19 (18.8 MB/s) - ‘small_cosmo_volume.hdf5’ saved [8396888/8396888]



# SWIFT - Running simulation

Dependencies
- HDF5
- MPI
- FFTW
- METIS
- GSL

#### Clone repository

In [2]:
git clone git@gitlab.cosma.dur.ac.uk:swift/swiftsim.git
cd swiftsim

Cloning into 'swiftsim'...
remote: Enumerating objects: 156627, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 156627 (delta 9), reused 4 (delta 4), pack-reused 156610 (from 1)
Receiving objects: 100% (156627/156627), 126.12 MiB | 23.66 MiB/s, done.
Resolving deltas: 100% (122594/122594), done.
Updating files: 100% (2448/2448), done.


#### Configure and compile

See https://swift.strw.leidenuniv.nl/docs/GettingStarted/compiling_code.html

Since HBTplus requires FOF information, we need to pass the `--enable-fof` flag

In [3]:
./autogen.sh
./configure --enable-fof

libtoolize: putting auxiliary files in AC_CONFIG_AUX_DIR, '.'.
libtoolize: linking file './ltmain.sh'
libtoolize: putting macros in AC_CONFIG_MACRO_DIRS, 'm4'.
libtoolize: linking file 'm4/libtool.m4'
libtoolize: linking file 'm4/ltoptions.m4'
libtoolize: linking file 'm4/ltsugar.m4'
libtoolize: linking file 'm4/ltversion.m4'
libtoolize: linking file 'm4/lt~obsolete.m4'
configure.ac:40: installing './compile'
configure.ac:242: installing './config.guess'
configure.ac:242: installing './config.sub'
configure.ac:25: installing './install-sh'
configure.ac:25: installing './missing'
Makefile.am: installing './INSTALL'
Makefile.am: installing './depcomp'
parallel-tests: installing './test-driver'
checking for a BSD-compatible install... /usr/bin/install -c
checking whether build environment is sane... yes
checking for a thread-safe mkdir -p... /usr/bin/mkdir -p
checking for gawk... gawk
checking whether make sets $(MAKE)... yes
checking whether make supports nested variables... yes
checking

In [4]:
make -j 4
cd ..

make  all-recursive
make[1]: Entering directory '/cosma/home/dp004/dc-mcgi1/SWIFTCON2024/swiftsim'
Making all in src
make[2]: Entering directory '/cosma/home/dp004/dc-mcgi1/SWIFTCON2024/swiftsim/src'
if test "X/usr/bin/git" != "X"; then \
    GIT_REVISION=`/usr/bin/git describe --abbrev=8  --always --tags --dirty 2>/dev/null`; \
    GIT_BRANCH=`/usr/bin/git branch 2>/dev/null | sed -n 's/^\* \(.*\)/\1/p' 2>/dev/null`; \
            GIT_DATE=`/usr/bin/git log -1 --format=%ci 2>/dev/null`; \
    sed -e "s,@PACKAGE_VERSION\@,1.0.0," \
        -e "s,@GIT_REVISION\@,${GIT_REVISION}," \
        -e "s|@GIT_BRANCH\@|${GIT_BRANCH}|" \
        -e "s|@GIT_DATE\@|${GIT_DATE}|" \
        -e "s|@SWIFT_CFLAGS\@|-flto -O3 -fomit-frame-pointer -malign-double -fstrict-aliasing -ffast-math -funroll-loops -march=znver2 -mavx2 -pthread -Wall -Wextra -Wno-unused-parameter -Wshadow -Werror -Wno-alloc-size-larger-than -Wstrict-prototypes|" version_string.h.in > version_string.h; \
else \
    if test ! -f vers

#### Set parameters

Point the code to the initial conditions we generated

Make sure to enable running the FOF before writing a snapshot

Set the number of output snapshots

In [24]:
#TODO: Cosmological parameters from ICS

In [5]:
cat > small_cosmo_volume_dm.yml << EOF
# Define the system of units to use internally. 
InternalUnitSystem:
  UnitMass_in_cgs:     1.98841e43    # 10^10 M_sun
  UnitLength_in_cgs:   3.08567758e24 # 1 Mpc
  UnitVelocity_in_cgs: 1e5           # 1 km/s
  UnitCurrent_in_cgs:  1             # Amperes
  UnitTemp_in_cgs:     1             # Kelvin

# WMAP9 cosmology
Cosmology:
  Omega_cdm:      0.2305
  Omega_lambda:   0.724
  Omega_b:        0.0455
  h:              0.703
  a_begin:        0.019607843	     # z_ini = 50.
  a_end:          1.0		         # z_end = 0.

# Parameters governing the time integration
TimeIntegration:
  dt_min:     1e-6 
  dt_max:     1e-2 

# Parameters for the self-gravity scheme
Gravity:
  eta:          0.025
  MAC:          adaptive
  theta_cr:     0.7
  epsilon_fmm:  0.001
  comoving_DM_softening:     0.0889     # 1/25th of the mean inter-particle separation: 88.9 kpc
  max_physical_DM_softening: 0.0889     # 1/25th of the mean inter-particle separation: 88.9 kpc
  mesh_side_length:       64

# Parameters governing the snapshots
Snapshots:
  basename:            snap
  delta_time:          1.15
  scale_factor_first:  0.1     # z = 9
  compression:         4
  invoke_fof:          1

# Parameters for running the friend-of-friends algorithm
FOF:  
  basename: fof_output        # Filename for the FOF outputs.
  min_group_size: 20          # The minimum no. of particles required for a group.
  linking_length_ratio: 0.2   # Linking length in units of the mean inter-particle separation
  seed_black_holes_enabled: 0
  linking_types: [0, 1, 0, 0, 0, 0, 0]         # Use DM as the primary FOF linking type
  attaching_types: [0, 0, 0, 0, 0, 0, 0]

# Parameters governing the conserved quantities statistics
Statistics:
  delta_time:          1.01
  scale_factor_first:  0.02

# Parameters governing the cell tree
Scheduler:
  max_top_level_cells: 8
  cell_split_size:     50
  
# Parameters related to the initial conditions
InitialConditions:
  file_name:                   small_cosmo_volume.hdf5
  periodic:                    1
  cleanup_h_factors:           1    
  cleanup_velocity_factors:    1
EOF

#### Run the code

In [8]:
./swiftsim/swift --cosmology --self-gravity --fof --threads=8 small_cosmo_volume_dm.yml

 Welcome to the cosmological hydrodynamical code
    ______       _________________
   / ___/ |     / /  _/ ___/_  __/
   \__ \| | /| / // // /_   / /   
  ___/ /| |/ |/ // // __/  / /    
 /____/ |__/|__/___/_/    /_/     
 SPH With Inter-dependent Fine-grained Tasking

 Version : 1.0.0
 Revision: v1.0.0-328-g6b2d9001, Branch: master, Date: 2024-09-15 15:44:45 +0000
 Webpage : www.swiftsim.com

 Config. options: '--enable-fof'

 Compiler: GCC, Version: 14.1.0
 CFLAGS  : '-flto -O3 -fomit-frame-pointer -malign-double -fstrict-aliasing -ffast-math -funroll-loops -march=znver2 -mavx2 -pthread -Wall -Wextra -Wno-unused-parameter -Wshadow -Werror -Wno-alloc-size-larger-than -Wstrict-prototypes'

 HDF5 library version     : 1.12.3
 FFTW library version     : 3.x (details not available)
 GSL library version      : 2.8

[00000.0] main: CPU frequency used for tick conversion: 2894525663 Hz
[00000.0] main: Running on: login8a.pri.cosma.local
[00000.0] main: sizeof(part)        is  160 bytes.
[0

#### Output of run

- `snap_xxxx.hdf5`
- `fof_output_xxxx.hdf5`
- `statistics.txt`
- `used_parameters.txt`
- `timesteps.txt`

In [25]:
ls

 cosma_env.sh             getIC.sh                    snap_0011.hdf5
 dependency_graph_0.csv   HBT_config.txt              snap_0012.hdf5
 fof_output_0000.hdf5     HBT_output                  snap_0013.hdf5
 fof_output_0001.hdf5     HBTplus                     snap_0014.hdf5
 fof_output_0002.hdf5     LICENSE                     snap_0015.hdf5
 fof_output_0003.hdf5     README.md                   snap_0016.hdf5
 fof_output_0004.hdf5     requirements.txt            snap_0017.hdf5
 fof_output_0005.hdf5     restart                     snap_0018.hdf5
 fof_output_0006.hdf5     small_cosmo_volume_dm.yml   snap.xmf
 fof_output_0007.hdf5     small_cosmo_volume.hdf5     SOAP
 fof_output_0008.hdf5     snap_0000.hdf5              SOAP_config.yml
 fof_output_0009.hdf5     snap_0001.hdf5              statistics.txt
 fof_output_0010.hdf5     snap_0002.hdf5             'SWIFT DMO pipeline.ipynb'
 fof_output_0011.hdf5     snap_0003.hdf5              swift.log
 fof_output_0012.hdf5     snap_0004.hdf5   

In [ ]:
# TODO Density Plot?

# HBTplus - Halo finding

Some description of HBT

Dependencies
- CMake
- HDF5
- MPI

#### Clone

In [9]:
git clone git@github.com:SWIFTSIM/HBTplus.git
cd HBTplus

Cloning into 'HBTplus'...
remote: Enumerating objects: 4653, done.
remote: Counting objects: 100% (1123/1123), done.
remote: Compressing objects: 100% (292/292), done.
remote: Total 4653 (delta 955), reused 911 (delta 831), pack-reused 3530 (from 1)
Receiving objects: 100% (4653/4653), 1.20 MiB | 3.15 MiB/s, done.
Resolving deltas: 100% (3439/3439), done.


#### Configure and compile

Use `ccmake` to see what options are available

In [10]:
cmake -B$PWD/build -D HBT_USE_OPENMP=ON -D HBT_DM_ONLY=ON -D HBT_UNSIGNED_LONG_ID_OUTPUT=OFF

-- The C compiler identification is GNU 14.1.0
-- The CXX compiler identification is GNU 14.1.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /cosma/local/gcc/14.1.0/bin/gcc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /cosma/local/gcc/14.1.0/bin/g++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Found MPI_C: /cosma/local/openmpi/gnu_14.1.0/5.0.3/lib/libmpi.so (found version "3.1") 
-- Found MPI_CXX: /cosma/local/openmpi/gnu_14.1.0/5.0.3/lib/libmpi.so (found version "3.1") 
-- Found MPI: TRUE (found version "3.1") found components: C CXX 
-- Found HDF5: /cosma/local/parallel-hdf5/gnu_14.1.0_omp

In [11]:
cd build
make -j 4
cd ../..

[  1%] Generating git_version_info.c
[  5%] Building CXX object unit_tests/CMakeFiles/testfuncs.dir/make_test_subhalo.cpp.o
[  5%] Building CXX object unit_tests/CMakeFiles/testfuncs.dir/verify.cpp.o
[  9%] Building CXX object CMakeFiles/hbtfuncs.dir/src/config_parser.cpp.o
[  9%] Building CXX object CMakeFiles/hbtfuncs.dir/src/geometric_tree.cpp.o
[ 11%] Building CXX object CMakeFiles/hbtfuncs.dir/src/gravity_tree.cpp.o
[ 11%] Built target testfuncs
[ 12%] Building CXX object CMakeFiles/hbtfuncs.dir/src/halo.cpp.o
[ 14%] Building CXX object CMakeFiles/hbtfuncs.dir/src/hdf_wrapper.cpp.o
[ 16%] Building CXX object CMakeFiles/hbtfuncs.dir/src/linkedlist_base.cpp.o
[ 18%] Building CXX object CMakeFiles/hbtfuncs.dir/src/linkedlist.cpp.o
[ 20%] Building CXX object CMakeFiles/hbtfuncs.dir/src/merger_tree.cpp.o
[ 22%] Building CXX object CMakeFiles/hbtfuncs.dir/src/mpi_wrapper.cpp.o
[ 24%] Building CXX object CMakeFiles/hbtfuncs.dir/src/mymath.cpp.o
[ 25%] Building CXX object CMakeFiles/hbtfu

#### Set parameters

In [12]:
cat > HBT_config.txt << EOF
# Configuration file used to run on small DMO simulation 

# Compulsary Params
SnapshotPath ./                      # Location of snapshots
SnapshotFileBase snap                # Basename of snapshot files
HaloPath ./                          # Location of FOF ID files (normal snapshot for SWIFT)
SubhaloPath HBT_output               # Output directory
# For the swiftsim reader these will be read in from snapshots automatically
BoxSize -1
SofteningHalo -1
MaxPhysicalSofteningHalo -1

# Reader
SnapshotFormat swiftsim
GroupFileFormat swiftsim_particle_index
MinSnapshotIndex 0                  # First snapshot to use
MaxSnapshotIndex 18                 # Final snapshot to use
MaxConcurrentIO 8                   # Number of cores for IO
MinNumPartOfSub 20                  # Minimum number of particles in a subhalo

# Units
MassInMsunh 6.81e9                  # Removes h factors from the final output 
LengthInMpch 0.681                  # Removes h factors from the final output
VelInKmS 1
EOF

##### Run

Run with a single rank as we will use 8 threads

In [13]:
export OMP_NUM_THREADS=8
mpirun -np 1 ./HBTplus/build/HBT HBT_config.txt

HBT compiled using git branch: master and commit: 29b4d4e41d2179b3f19a01eb07d6b650793a0ae4 (clean)
Reading configuration file HBT_config.txt
Running ./HBTplus/build/HBT from snapshot 0 to 18 using configuration file HBT_config.txt
./HBTplus/build/HBT run using 1 mpi tasks, each with 8 threads
Configured with the following data type sizes (bytes):
  Real quantities    : 4
  Integer quantities : 8
  Particle velocities: 4
  Particle masses    : 4
  Size of Particle_t : 48
Skipping empty snapshot -1

262144 particles loaded at Snapshot 0(0)
0 groups loaded at snapshot 0(0)
Unbinding...
saving 0 subhalos to HBT_output/000
SnapshotIndex 0 done. It took 0.297505 seconds.

262144 particles loaded at Snapshot 1(1)
0 groups loaded at snapshot 1(1)
Unbinding...
saving 0 subhalos to HBT_output/001
SnapshotIndex 1 done. It took 0.291474 seconds.

262144 particles loaded at Snapshot 2(2)
0 groups loaded at snapshot 2(2)
Unbinding...
saving 0 subhalos to HBT_output/002
SnapshotIndex 2 done. It took 

#### Output of run

For each snapshot each HBT rank outputs two files.
- The `SubSnap_xxx.y.hdf5` files contain information about the bound subhalos identified by HBTplus, including their particles
- The `SrcSnap_xxx.y.hdf5` files contain a list of the particles associated with each subhalo, but which are not bound at the current snapshot. These files are only used if HBT if restarted, and so can be deleted once halo finding has been completed

In [14]:
echo "Contents of HBT_output:"
ls HBT_output
echo "Output for snapshot 18:"
ls HBT_output/018

Contents of HBT_output:
000  002  004  006  008  010  012  014  016  018             timing.log
001  003  005  007  009  011  013  015  017  Parameters.log
Output for snapshot 18:
SrcSnap_018.0.hdf5  SubSnap_018.0.hdf5


# SOAP - Calculating halo properties

Dependencies
- mpi4py
- h5py built with parallel HDF5
- Standard python modules (see requirements.txt)

#### Clone repo

In [15]:
git clone git@github.com:SWIFTSIM/SOAP.git

Cloning into 'SOAP'...
remote: Enumerating objects: 4447, done.
remote: Counting objects: 100% (2277/2277), done.
remote: Compressing objects: 100% (698/698), done.
remote: Total 4447 (delta 1782), reused 1946 (delta 1572), pack-reused 2170 (from 1)
Receiving objects: 100% (4447/4447), 1.84 MiB | 4.04 MiB/s, done.
Resolving deltas: 100% (3045/3045), done.


In [21]:
cat > SOAP_config.yml << EOF
#### Parameter file

# Values in this section are substituted into the other sections
# The simulation name (box size and resolution) and snapshot will be appended
# to these to get the full name of the input/output files/directories
Parameters:
  sim_dir: /cosma/home/dp004/dc-mcgi1/swift_workshop
  output_dir: /cosma/home/dp004/dc-mcgi1/swift_workshop
  scratch_dir: /cosma/home/dp004/dc-mcgi1/swift_workshop

# Location of the Swift snapshots:
Snapshots:
  # Use {snap_nr:04d} for the snapshot number and {file_nr} for the file number.
  filename: "{sim_dir}/snap_{snap_nr:04d}.hdf5"

# Which halo finder we're using, and base name for halo finder output files
HaloFinder:
  type: HBTplus
  filename: "{sim_dir}/HBT_output/{snap_nr:03d}/SubSnap_{snap_nr:03d}"
  fof_filename: "{sim_dir}/fof_output_{snap_nr:04d}.hdf5"

GroupMembership:
  # Where to write the group membership files
  filename: "{output_dir}/SOAP_output/membership_{snap_nr:04d}.hdf5"

HaloProperties:
  # Where to write the halo properties file
  filename: "{output_dir}/SOAP_output/halo_properties_{snap_nr:04d}.hdf5"
  # Where to write temporary chunk output
  chunk_dir: "{scratch_dir}/SOAP-tmp/{halo_finder}/"

ApertureProperties:
  properties:
    {}
  variations:
    {}
ProjectedApertureProperties:
  properties:
    {}
  variations:
    {}
SOProperties:
  properties:
    CentreOfMass: true
    CentreOfMassVelocity: true
    Concentration: true
    MassFractionSatellites: true
    MassFractionExternal: true
    NumberOfDarkMatterParticles: true
    SORadius: true
    SpinParameter: true
    TotalMass: true
  variations:
    200_crit:
      type: crit
      value: 200.0
    200_mean:
      type: mean
      value: 200.0
    500_crit:
      type: crit
      value: 500.0
SubhaloProperties:
  properties:
    CentreOfMass: true
    CentreOfMassVelocity: true
    NumberOfDarkMatterParticles: true
    MaximumCircularVelocity: true
    MaximumCircularVelocityUnsoftened: true
    MaximumCircularVelocityRadiusUnsoftened: true
    SpinParameter: true
    TotalMass: true
  variations:
    Bound:
      bound_only: true
filters:
  general:
    limit: 100
    properties:
      - BoundSubhalo/NumberOfGasParticles
      - BoundSubhalo/NumberOfDarkMatterParticles
      - BoundSubhalo/NumberOfStarParticles
      - BoundSubhalo/NumberOfBlackHoleParticles
    combine_properties: sum
  baryon:
    limit: 100
    properties:
      - BoundSubhalo/NumberOfGasParticles
      - BoundSubhalo/NumberOfStarParticles
    combine_properties: sum
  dm:
    limit: 100
    properties:
      - BoundSubhalo/NumberOfDarkMatterParticles
  gas:
    limit: 100
    properties:
      - BoundSubhalo/NumberOfGasParticles
  star:
    limit: 100
    properties:
      - BoundSubhalo/NumberOfStarParticles
calculations:
  recalculate_xrays: false
  calculate_missing_properties: false
  min_read_radius_cmpc: 5
  recently_heated_gas_filter:
    delta_time_myr: 15
    use_AGN_delta_T: true
EOF

In [22]:
mpirun -np 8 $python SOAP/group_membership.py \
    --sim-name=DM_test \
    --snap-nr=18 \
    SOAP_config.yml

lfs setstripe: setstripe error for '/cosma/home/dp004/dc-mcgi1/swift_workshop/SOAP_output': Inappropriate ioctl for device
Number of group particle IDs per rank min=0, max=63049
Calculating group membership for type PartType1
  Number of snapshot particle IDs per rank min=32768, max=32768
  Matching SWIFT particle IDs to bound IDs
  Assigning bound group membership to SWIFT particles
  Assigning rank by binding energy to SWIFT particles
  Writing out group membership of SWIFT particles
Done.


In [23]:
mpirun -np 8 $python SOAP/compute_halo_properties.py \
    --sim-name=DM_test \
    --snap-nr=18 \
    --chunks=1 \
    --dmo \
    SOAP_config.yml

Starting halo properties calculation on 8 MPI ranks
Can process 1 chunks in parallel using 8 ranks per chunk
Number of MPI ranks per node reading snapshots: 32
Halo format is HBTplus
Halo basename is /cosma/home/dp004/dc-mcgi1/swift_workshop/HBT_output/{snap_nr:03d}/SubSnap_{snap_nr:03d}
Output file is /cosma/home/dp004/dc-mcgi1/swift_workshop/SOAP_output/halo_properties_{snap_nr:04d}.hdf5
Snapshot number is 18
Could not find lightcone observer position in snapshot file. Defaulting to centre of box.
Halo property calculations enabled:
  bound_subhalo_properties
  SO_200_crit
  SO_200_mean
  SO_500_crit
for central and satellite halos
Properties not present in the parameter file will not be calculated
Run in DMO mode
Category filters :
  general   100
  dm        100
  gas       100
  star      100
  baryon    100
lfs setstripe: setstripe error for '/cosma/home/dp004/dc-mcgi1/swift_workshop/SOAP_output': Inappropriate ioctl for device
Using Peano domain decomposition with bits=10
Readin

#### SOAP output

#### Compression